In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('../data/ag_news_csv/test.csv', header=None)

In [3]:
from tqdm.autonotebook import tqdm

/home/fadeich/env3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """Entry point for launching an IPython kernel.


In [4]:
from model import CharCNN
import torch
from torch.autograd import Variable
from metric import print_f_score
import torch.nn.functional as F

In [5]:
import json
with open('../alphabet.json') as f:
    res = json.load(f)
num_features = len(res)

In [6]:
class Args():
    num_features = num_features
    dropout = 0.5
    test_path = '../data/ag_news_csv/test.csv'
    alphabet_path = '../alphabet.json'
    batch_size = 20
    num_workers = 4
args = Args()

In [7]:
char_model = CharCNN(args)
checkpoint = torch.load('models/CharCNN_best.pth.tar')
char_model.load_state_dict(checkpoint['state_dict'])
char_model = char_model.to("cuda:0")

In [8]:
def test_model(test_dataset, model, limit=None):
    corrects, avg_loss, accumulated_loss, size, fooled = 0, 0, 0, 0, 0
    predicates_all, target_all = [], []
    for i_batch, (data) in enumerate(tqdm(test_dataset)):
        inputs_init, inputs, target = data
        target.sub_(1)
        size+=1
        inputs_init = Variable(torch.unsqueeze(inputs_init, 0).to("cuda:0"))
        inputs = Variable(torch.unsqueeze(inputs, 0).to("cuda:0"))
        target = Variable(torch.unsqueeze(target, 0).to("cuda:0"))
        logit_init = model(inputs_init)
        logit = model(inputs)
        fooled += (logit_init.argmax() != logit.argmax()).detach().cpu().numpy()
    
        predicates = torch.max(logit, 1)[1].view(target.size()).data
        accumulated_loss += F.nll_loss(logit, target, size_average=False).data.item()
        corrects += (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum().item()
        predicates_all+=predicates.cpu().numpy().tolist()
        target_all+=target.data.cpu().numpy().tolist()
        if limit is not None and i_batch == limit:
            break

    avg_loss = accumulated_loss/size
    accuracy = 100.0 * corrects/size
    fooled = 100.0 * fooled/size
    print('\rEvaluation - loss: {:.6f}  acc: {:.3f}%({}/{}) fooling rate: {:.3f}'.format(avg_loss, 
                                                                       accuracy, 
                                                                       corrects, 
                                                                       size,
                                                                       fooled))
    print_f_score(predicates_all, target_all)
    return accuracy, fooled, predicates_all, target_all

In [9]:
from data_loader_hotflip_greedy import AGNEWs_HotFlip_Greedy

In [10]:
test_dataset = AGNEWs_HotFlip_Greedy(per_corrupt=1, model=char_model, theta=0.5, dpp=False,
                                  label_data_path=args.test_path, alphabet_path=args.alphabet_path)
accuracy, fooled, predicates_all, target_all = test_model(test_dataset, char_model, limit=10)

Evaluation - loss: 0.621648  acc: 81.818%(9/11) fooling rate: 18.182
Label:   2	Prec:  50.0% (1/2)        Recall: 100.0% (1/1)        F-Score:  66.7%
Label:   3	Prec: 100.0% (8/8)        Recall:  80.0% (8/10)       F-Score:  88.9%


/home/fadeich/texts/char-cnn-text-classification-pytorch/final_code/model.py:75: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.log_softmax(x)
/home/fadeich/env3/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [11]:
test_dataset.print_string(0)

original: fears for t n pension after talks unions representing workers at turner   newall say they are 'disappointed' after talks with stricken parent firm federal mogul.
corrupt: fears for t n pension after talks unions representing workers at turner   newall say they are 'disappointed' afte= talks with stricken parent firm federal mogul.

In [12]:
from data_loader_hotflip_beam import AGNEWs_HotFlip_Beam

In [13]:
test_dataset_beam = AGNEWs_HotFlip_Beam(per_corrupt=1, model=char_model, theta=0.5, k=1, dpp=False,
                                        label_data_path=args.test_path, alphabet_path=args.alphabet_path)
accuracy, fooled, predicates_all, target_all = test_model(test_dataset_beam, char_model, limit=10)


Evaluation - loss: 0.884745  acc: 63.636%(7/11) fooling rate: 36.364
Label:   2	Prec:  25.0% (1/4)        Recall: 100.0% (1/1)        F-Score:  40.0%
Label:   3	Prec: 100.0% (6/6)        Recall:  60.0% (6/10)       F-Score:  75.0%


In [14]:
test_dataset_beam.print_string(0)

original: fears for t n pension after talks unions representing workers at turner   newall say they are 'disappointed' after talks with stricken parent firm federal mogul.
corrupt: fears for t n pension after talks unions representing workers at turner   newall say they are 'disappointed' afte= talks with stricken parent firm federal mogul.

In [15]:
from data_loader_deepwordbug import WordBug

In [16]:
test_dataset_beam = WordBug(per_corrupt=4, model=char_model, scoring='beam',
                                        label_data_path=args.test_path, alphabet_path=args.alphabet_path)
accuracy, fooled, predicates_all, target_all = test_model(test_dataset_beam, char_model, limit=10)

Evaluation - loss: 2.176774  acc: 36.364%(4/11) fooling rate: 63.636
Label:   2	Prec:   0.0% (0/6)        Recall:   0.0% (0/1)        F-Score:   N/A
Label:   3	Prec: 100.0% (4/4)        Recall:  40.0% (4/10)       F-Score:  57.1%


In [17]:
test_dataset_beam = WordBug(per_corrupt=4, model=char_model, scoring='greedy',
                                        label_data_path=args.test_path, alphabet_path=args.alphabet_path)
accuracy, fooled, predicates_all, target_all = test_model(test_dataset_beam, char_model, limit=10)

Evaluation - loss: 1.300553  acc: 45.455%(5/11) fooling rate: 54.545
Label:   2	Prec:   0.0% (0/5)        Recall:   0.0% (0/1)        F-Score:   N/A
Label:   3	Prec: 100.0% (5/5)        Recall:  50.0% (5/10)       F-Score:  66.7%


In [18]:
from data_loader_hotflip_targeted import AGNEWs_HotFlip_Targeted

In [19]:
test_dataset_beam = AGNEWs_HotFlip_Targeted(combine=False, per_corrupt=1, model=char_model, k=10,
                                            label_data_path=args.test_path, alphabet_path=args.alphabet_path)

In [20]:
accuracy, fooled, predicates_all, target_all = test_model(test_dataset_beam, char_model, limit=20)

Evaluation - loss: 3.318901  acc: 14.286%(3/21) fooling rate: 28.571
Label:   0	Prec: 100.0% (1/1)        Recall:   8.3% (1/12)       F-Score:  15.4%
Label:   1	Prec: 100.0% (1/1)        Recall:  50.0% (1/2)        F-Score:  66.7%
Label:   2	Prec:  25.0% (1/4)        Recall:  16.7% (1/6)        F-Score:  20.0%
Label:   3	Prec:   0.0% (0/15)       Recall:   0.0% (0/1)        F-Score:   N/A


In [21]:
test_dataset_beam = AGNEWs_HotFlip_Targeted(combine=True, per_corrupt=1, model=char_model, k=10,
                                            label_data_path=args.test_path, alphabet_path=args.alphabet_path)

In [22]:
accuracy, fooled, predicates_all, target_all = test_model(test_dataset_beam, char_model, limit=20)

Evaluation - loss: 3.211607  acc: 14.286%(3/21) fooling rate: 33.333
Label:   0	Prec: 100.0% (2/2)        Recall:  16.7% (2/12)       F-Score:  28.6%
Label:   1	Prec:   0.0% (0/0)        Recall:   0.0% (0/2)        F-Score:   N/A
Label:   2	Prec:  14.3% (1/7)        Recall:  16.7% (1/6)        F-Score:  15.4%
Label:   3	Prec:   0.0% (0/12)       Recall:   0.0% (0/1)        F-Score:   N/A
